In [9]:
import altair as alt
import pandas as pd
from IPython.display import display

nvidia_csv_path = "C:/Users/bruno/Code/pixelweave/build/benchmark/benchmark.csv"
mac_csv_path = "D:/Downloads/benchmarkM2.csv"

def load_data(path):
    csv_data = pd.read_csv(path)
    return csv_data[(csv_data["InputWidth"] == csv_data["OutputWidth"])]

nvidia_data = load_data(nvidia_csv_path)
mac_data = load_data(mac_csv_path)

In [10]:
def create_chart(df, name):
    input_formats = df["InputFormat"].unique()
    input_formats_dropdown = alt.binding_select(options=input_formats, name="Input format")
    input_formats_select = alt.selection_single(
        fields=["InputFormat"],
        bind=input_formats_dropdown,
        name="Input format",
        init={'InputFormat': input_formats[1]}
    )

    output_formats = df["OutputFormat"].unique()
    output_formats_dropdown = alt.binding_select(options=output_formats, name="Output format")
    output_formats_select = alt.selection_single(
        fields=["OutputFormat"],
        bind=output_formats_dropdown,
        name="Output format",
        init={'OutputFormat': output_formats[1]}
    )

    columns = [
        "CopyToDeviceVisibleTimeMicros:Q",
        "TransferDeviceVisibleToDeviceLocalTimeMicros:Q",
        "ComputeConversionTimeMicros:Q",
        "CopyDeviceVisibleToHostLocalTimeMicros:Q",
    ]

    colors = [
        'navy', 'chocolate', 'darkseagreen', 'fuchsia'
    ]

    base = alt.Chart(df).mark_area(size=1).encode(
        x=alt.X("OutputHeight:Q", title="Resolution (Height, 16:9)"),
        y=alt.Y("TotalTime:Q", title="Time (Micros)"),
        opacity=alt.value(0.3)
    )

    layers = [
        alt.Chart(df).mark_line().encode(
            x=alt.X("OutputHeight:Q"),
            y=alt.Y(column),
            tooltip=[column],
            color=alt.value(color)
        )
        for column, color in zip(columns, colors)
    ]

    return alt.layer(
        *[base, *layers]
    ).resolve_scale(
        y="shared",
        color="independent"
    ).add_selection(
        input_formats_select, output_formats_select
    ).transform_filter(
        input_formats_select & output_formats_select
    ).properties(
        title=name
    ).interactive()

nvidia_chart = create_chart(nvidia_data, "Nvidia")
mac_chart = create_chart(mac_data, "Mac")

alt.concat(nvidia_chart, mac_chart).resolve_scale(y="shared")

alt.ConcatChart(...)